In [ ]:
import pandas as pd 
import general_utils
import holidays
import matplotlib.pyplot as plt
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter
import matplotlib.colors as mcolors
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import xlsxwriter
import io
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


%load_ext autoreload
%autoreload 2

In [ ]:
directory = 'C:/Users/maran/OneDrive/Documents/Git Profile/Data-Projects/CAVU/raw_data'
bookings_sample_file = 'bookings_sample (1).csv'
flights_sample_file = 'flights_sample.csv'
bookings_sample_path = directory + '/' + bookings_sample_file
flights_sample_path = directory + '/' + flights_sample_file
final_bookings_file = 'final_bookings_df.csv'
directorys_to_make = []
csv_directory = 'temp_csv/'
reports_directory = 'reports/'
os.makedirs(reports_directory, exist_ok=True)
os.makedirs(csv_directory, exist_ok=True)
use_weekly = True

path_to_final_bookings = csv_directory + final_bookings_file

car_park_columns = ['carpark1', 'carpark2', 'carpark3', 'carpark4', 'carpark5', 'carpark6', 'carpark7']

second_last_ledger_column_names = ['date_occupied', 'terminal', 'carpark_name', 'occupancy','total_revenue', 'bookings_count', 'avg_duration_days', 'min_duration_days', 'avg_lead_time_days', 'min_lead_time_days',
       'discounted_bookings', 'discount_ratio', 'cancel_ratio', 'dow', 'month', 'is_weekend', 'flight_date', 'terminal_name', 'total_flights',
       'total_passengers', 'intl_flights', 'intl_share', 'europe_share', 'asia_share', 'na_share', 'africa_share', 'holiday_flag', 'weekofyear']
       
new_second_ledger_column_time_name = 'uc_local_flight_date'
second_last_ledger_column_names.append(new_second_ledger_column_time_name)

version = 'v0.1.0'

naive_split_bool = False

if use_weekly:
    report_name = "CAVU_Analysis_Report_{version}_Weekly"
else:
    report_name = "CAVU_Analysis_Report_{version}_Daily"

path_to_report = f"{reports_directory}/{report_name}.xlsx"

# 1. Load and clean data

In [ ]:
raw_bookings_df = pd.read_csv(bookings_sample_path)
raw_flights_df = pd.read_csv(flights_sample_path)

## Flights data Prep

In [ ]:
flights_df = raw_flights_df.copy(deep=True)
flights_df.head()

In [ ]:
flights_df.isna().sum()

In [ ]:
flights_df['destination_airport_name_continent'].unique()

In [ ]:
flights_df[flights_df['destination_airport_name_continent'] == 'Unknown Continent']

flights_df['flight_number_full_icao'] = flights_df['flight_number_full_icao'].str.upper()
flights_df['is_international'] = flights_df['destination_airport_dom_intern'] == 'International'
flights_df['is_europe'] = flights_df['destination_airport_name_continent'] == 'Europe'
flights_df['is_asia'] = flights_df['destination_airport_name_continent'] == 'Asia'

manohar_rows = flights_df['destination_airport_name'].str.contains('Manohar', na=False)
flights_df.loc[manohar_rows, 'is_asia'] = True

flights_df['is_north_america'] = flights_df['destination_airport_name_continent'] == 'North America'
flights_df['is_africa'] = flights_df['destination_airport_name_continent'] == 'Africa'

flights_df.shape

In [ ]:
flight_min_date = flights_df['flight_date'].min()
flight_max_date = flights_df['flight_date'].max()

print(f"Flight Date Range: {flight_min_date} to {flight_max_date}")

## Bookings data prep

In [ ]:
raw_booking_records = raw_bookings_df.shape[0]
raw_booking_records

In [ ]:
raw_bookings_df.isna().sum()

In [ ]:
raw_bookings_df['started_at'].min(), raw_bookings_df['started_at'].max(), raw_bookings_df['created_at'].min(), raw_bookings_df['created_at'].max(), raw_bookings_df['closed_at'].min(), raw_bookings_df['closed_at'].max()

In [ ]:
raw_bookings_df[raw_bookings_df['created_at'] == '2018-09-09 15:50:21.487'].shape
raw_bookings_df[raw_bookings_df.duplicated(subset=['created_at', 'amount'])].head(2)

In [ ]:
raw_bookings_df[raw_bookings_df.duplicated(subset=['created_at', 'outbound_flight_no'])]

In [ ]:
bookings_df = raw_bookings_df.copy(deep=True)
bookings_df.head()

In [ ]:
bookings_df['booking_status'].value_counts()

In [ ]:
confirmed_booking = bookings_df['booking_status'] == 'Confirmed'

same_time_period = bookings_df['created_at'].between(flight_min_date, flight_max_date, inclusive='both')

bookings_df.loc[:, 'to_keep'] = confirmed_booking & same_time_period

bookings_df['to_keep'].fillna(False, inplace=True)

filtered_bookings_df = bookings_df[bookings_df['to_keep']].drop(columns=['to_keep'])

filtered_bookings_df.head()

In [ ]:
filtered_bookings_df[~filtered_bookings_df['started_at'].astype(str).str.endswith('+00')].shape, filtered_bookings_df[~filtered_bookings_df['closed_at'].astype(str).str.endswith('+00')].shape

In [ ]:
filtered_bookings_df['created_at'] = pd.to_datetime(filtered_bookings_df['created_at'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
filtered_bookings_df['started_at'] = pd.to_datetime(filtered_bookings_df['started_at'], format='%Y-%m-%d %H:%M:%S%z', errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
filtered_bookings_df['closed_at'] = pd.to_datetime(filtered_bookings_df['closed_at'], format='%Y-%m-%d %H:%M:%S%z', errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
filtered_bookings_df['is_discount_booking'] = filtered_bookings_df['is_discount_booking'].astype(bool)
filtered_bookings_df['booking_status'] = filtered_bookings_df['booking_status'].astype('category')

filtered_bookings_df['flight_date'] = pd.to_datetime(filtered_bookings_df['started_at'], errors='coerce').dt.strftime('%Y-%m-%d')

filtered_bookings_df['created_at_dt'] = pd.to_datetime(filtered_bookings_df['created_at'], errors='coerce')
filtered_bookings_df['started_at_dt'] = pd.to_datetime(filtered_bookings_df['started_at'], errors='coerce')
filtered_bookings_df['closed_at_dt'] = pd.to_datetime(filtered_bookings_df['closed_at'], errors='coerce')

filtered_bookings_df['parking_duration_days'] = (filtered_bookings_df['closed_at_dt'] - filtered_bookings_df['started_at_dt']).dt.days
filtered_bookings_df['lead_time_days'] = (filtered_bookings_df['started_at_dt'] - filtered_bookings_df['created_at_dt']).dt.days

filtered_bookings_df['flight_number_full_icao'] = filtered_bookings_df['outbound_flight_no'].str.upper()

filtered_bookings_df.head()


In [ ]:
# Analysis for bookings_df
nan_terminal_records_bookings = bookings_df[bookings_df['terminal'].isnull()].shape[0]
total_raw_booking_records_bookings = bookings_df.shape[0]
nan_terminal_percentage_bookings = (nan_terminal_records_bookings / total_raw_booking_records_bookings) * 100
total_records_to_keep_bookings = bookings_df[bookings_df['to_keep']].shape[0]

print("bookings_df:")
print(f"  Total records: {total_raw_booking_records_bookings}")
print(f"  Records to keep: {total_records_to_keep_bookings}")
print(f"  NaN 'terminal': {nan_terminal_records_bookings}")
print(f"  % NaN 'terminal': {nan_terminal_percentage_bookings:.2f}%")
print(f"  NaN 'terminal' & to_keep: {bookings_df[bookings_df['terminal'].isnull() & bookings_df['to_keep']].shape[0]}")
print(f"  % NaN 'terminal' & to_keep: {(bookings_df[bookings_df['terminal'].isnull() & bookings_df['to_keep']].shape[0] / total_records_to_keep_bookings) * 100:.2f}%")

# Analysis for filtered_bookings_df
nan_terminal_records_filtered = filtered_bookings_df[filtered_bookings_df['terminal'].isnull()].shape[0] if 'terminal' in filtered_bookings_df.columns else 0
total_raw_booking_records_filtered = filtered_bookings_df.shape[0]
nan_terminal_percentage_filtered = (nan_terminal_records_filtered / total_raw_booking_records_filtered) * 100 if total_raw_booking_records_filtered > 0 else 0

print("\nfiltered_bookings_df:")
print(f"  Total records: {total_raw_booking_records_filtered}")
print(f"  NaN 'terminal': {nan_terminal_records_filtered}")
print(f"  % NaN 'terminal': {nan_terminal_percentage_filtered:.2f}%")

In [ ]:
has_terminal_bookings_df = filtered_bookings_df[~filtered_bookings_df['terminal'].isna()]

missing_terminal_bookings_df = filtered_bookings_df[filtered_bookings_df['terminal'].isna()]#.rename(columns={'outbound_flight_no': 'flight_number_full_icao'})

required_flight_cols = ['flight_number_full_icao', 'terminal_name', 'flight_date']

recovered_terminal_flights_df = missing_terminal_bookings_df.merge(flights_df[required_flight_cols], on=['flight_number_full_icao', 'flight_date'], how='left')

recovered_terminal_flights_df = recovered_terminal_flights_df.drop(columns=['terminal']).rename(columns={'terminal_name': 'terminal'})

bookings_validated_df = pd.concat([has_terminal_bookings_df, recovered_terminal_flights_df], ignore_index=True)

nan_terminals = bookings_validated_df['terminal'].isna().sum()

bookings_validated_df = bookings_validated_df[bookings_validated_df['terminal'].notna()]

# bookings_validated_df['terminal'] = bookings_validated_df['terminal'].fillna('Unknown')

bookings_validated_df.shape, bookings_df.shape, nan_terminals

In [ ]:
final_bookings_df = bookings_validated_df.melt(
    id_vars=[c for c in bookings_validated_df.columns if c not in car_park_columns],
    value_vars=car_park_columns,
    var_name="carpark_name",
    value_name="is_carpark"
)

final_bookings_df = final_bookings_df[(final_bookings_df['is_carpark'] == 1).drop(columns=['is_carpark'])].reset_index(drop=True)
final_bookings_df.head()

# 2. Aggregate for features

## Daily Bookings

In [ ]:
# Precompute normalized dates CORRECT AND SLIGHTLY FASTER
starts = final_bookings_df['started_at_dt'].dt.normalize()
ends = final_bookings_df['closed_at_dt'].dt.normalize()

# Generate date ranges using list comprehension (faster than apply)
final_bookings_df['occupancy_days'] = [
    pd.date_range(s, e, freq='D') 
    for s, e in zip(starts, ends)
]

In [ ]:
if os.path.exists(path_to_final_bookings):
    print("Loading final_bookings_df from CSV...")
    final_bookings_exploded_df = pd.read_csv(path_to_final_bookings)
else:
    print("Generating final_bookings_exploded_df and saving to CSV...")
    final_bookings_exploded_df = final_bookings_df.explode('occupancy_days').rename(columns={'occupancy_days': 'date_occupied'})
    final_bookings_exploded_df.to_csv(path_to_final_bookings, index=False)

In [ ]:
daily_bookings_df = final_bookings_exploded_df.groupby(['date_occupied', 'terminal', 'carpark_name']).agg(**{
    'occupancy': ('date_occupied', 'count'),
    'total_revenue': ('amount', 'sum'),
    'bookings_count': ('amount', 'size'),
    'avg_duration_days': ('parking_duration_days', 'mean'),
    'min_duration_days': ('parking_duration_days', 'min'),
    'avg_lead_time_days': ('lead_time_days', 'mean'),
    'min_lead_time_days': ('lead_time_days', 'min'),
    'discounted_bookings': ('is_discount_booking', 'sum'),
    'discount_ratio': ('is_discount_booking', 'mean'),
    'cancel_ratio': ('booking_status', lambda x: (x == 'canceled').mean())
}).reset_index()

daily_bookings_df['dow'] = pd.to_datetime(daily_bookings_df['date_occupied']).dt.dayofweek
daily_bookings_df['month'] = pd.to_datetime(daily_bookings_df['date_occupied']).dt.month
daily_bookings_df['is_weekend'] = daily_bookings_df['dow'].isin([5, 6]).astype(int)

daily_bookings_df.rename(columns={'date_occupied': new_second_ledger_column_time_name}, inplace=True)

daily_bookings_df[new_second_ledger_column_time_name] = pd.to_datetime(daily_bookings_df[new_second_ledger_column_time_name], errors='coerce')

daily_bookings_df

## Flights features

In [ ]:
flights_agg_df = flights_df.groupby(['flight_date', 'terminal_name']).agg(**{
    'total_flights': ('flight_number_full_icao', 'nunique'),
    'total_passengers': ('pax_quantity', 'sum'),
    'intl_flights': ('is_international', 'sum'),
    'intl_pax': ('pax_quantity', lambda x: x[flights_df.loc[x.index, 'is_international'] == 1].sum()),
    'europe_pax': ('pax_quantity', lambda x: x[flights_df.loc[x.index, 'is_europe'] == True].sum()),
    'asia_pax': ('pax_quantity', lambda x: x[flights_df.loc[x.index, 'is_asia'] == True].sum()),
    'na_pax': ('pax_quantity', lambda x: x[flights_df.loc[x.index, 'is_north_america'] == True].sum()),
    'africa_pax': ('pax_quantity', lambda x: x[flights_df.loc[x.index, 'is_africa'] == True].sum()),
}).reset_index()

flights_agg_df['flight_date'] = pd.to_datetime(flights_agg_df['flight_date'])

flights_agg_df['intl_share'] = flights_agg_df['intl_pax'] / flights_agg_df['total_passengers']
flights_agg_df['europe_share'] = flights_agg_df['europe_pax'] / flights_agg_df['total_passengers']
flights_agg_df['asia_share'] = flights_agg_df['asia_pax'] / flights_agg_df['total_passengers']
flights_agg_df['na_share'] = flights_agg_df['na_pax'] / flights_agg_df['total_passengers']
flights_agg_df['africa_share'] = flights_agg_df['africa_pax'] / flights_agg_df['total_passengers']

uk_holidays = holidays.UnitedKingdom(years=range(2017, 2026))
flights_agg_df['holiday_flag'] = flights_agg_df['flight_date'].dt.date.astype('datetime64[ns]').isin(uk_holidays).astype(int)

flights_agg_df['weekofyear'] = flights_agg_df['flight_date'].dt.isocalendar().week

flights_agg_df['financial_quarter'] = flights_agg_df['flight_date'].dt.quarter

flights_agg_df.rename(columns={'flight_date': new_second_ledger_column_time_name}, inplace=True)

flights_agg_df

In [ ]:
# daily_bookings_df.rename(columns={'date_occupied': new_second_ledger_column_time_name}, inplace=True)

flights_agg_df['uc_local_flight_date'].min(), flights_agg_df['uc_local_flight_date'].max(), flights_df['flight_date'].min(), flights_df['flight_date'].max()

## Create Master Frames

In [ ]:
merged_df = daily_bookings_df.merge(
    flights_agg_df,
    left_on=[new_second_ledger_column_time_name, 'terminal'],
    right_on=[new_second_ledger_column_time_name, 'terminal_name'],
    how='left'
)

In [ ]:
merged_df = daily_bookings_df.merge(flights_agg_df, left_on=[new_second_ledger_column_time_name, 'terminal'], right_on=[new_second_ledger_column_time_name, 'terminal_name'], how='left')
filtered_dates_rows = merged_df[new_second_ledger_column_time_name].between(flight_min_date, flight_max_date, inclusive='both')
master_df = merged_df.loc[filtered_dates_rows, :].drop(columns=['terminal_name'])

master_df.shape, merged_df.shape

## Filtering, cleaning and encoding

In [ ]:
to_nan_cols = [
    'total_flights', 'total_passengers', 'intl_share', 'europe_share', 'asia_share',
    'na_share', 'africa_share', 'holiday_flag', 'dow', 'month', 'is_weekend'
]

columns_to_encode = ['terminal', 'carpark_name', 'uc_local_flight_date']

encoded_columns_list = [f"{col}_num" for col in columns_to_encode]

master_df.dtypes

In [ ]:
master_df[to_nan_cols] = master_df[to_nan_cols].fillna(0)

for col in columns_to_encode:
    master_df[f"{col}_num"] = master_df[col].astype('category').cat.codes

## Weekly aggregation

In [ ]:
weekly_df = master_df.groupby(['weekofyear', 'terminal', 'carpark_name']).agg({
    'occupancy': 'sum',
    'total_revenue': 'sum',
    'bookings_count': 'sum',
    'avg_duration_days': 'mean',
    'min_duration_days': 'min',
    'avg_lead_time_days': 'mean',
    'min_lead_time_days': 'min',
    'discounted_bookings': 'sum',
    'discount_ratio': 'mean',
    'cancel_ratio': 'mean',
    'dow': 'first',
    'month': 'first',
    'is_weekend': 'first',
    'total_flights': 'sum',
    'total_passengers': 'sum',
    'intl_flights': 'sum',
    'intl_share': 'mean',
    'europe_share': 'mean',
    'asia_share': 'mean',
    'na_share': 'mean',
    'africa_share': 'mean',
    'holiday_flag': 'max',
    'carpark_name_num': 'first',
    'financial_quarter': 'first',
    'terminal_num': 'first'
}).reset_index().drop(columns=['terminal','carpark_name'])

weekly_df.shape

Naive

In [ ]:
weekly_df.dropna(inplace=True)

# 3. Explore Data

In [ ]:
master_df.columns

In [ ]:
sliced_flights_df = flights_df[flights_df['flight_date'].between(flight_min_date, flight_max_date, inclusive='both')].sort_values(by=['flight_date', 'terminal_name'])
sliced_flights_df['weekofyear'] = pd.to_datetime(sliced_flights_df['flight_date']).dt.isocalendar().week
sliced_flights_df.head()

In [ ]:
ax = sliced_flights_df.groupby(['weekofyear','terminal_name']).agg({'pax_quantity': 'sum'}).unstack().droplevel(level=0, axis=1).plot(
    kind='line', title='Weekly Passenger Count Over Time', figsize=(12,6)
)
ax.set_ylim(bottom=0)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x):,}"))
ax.grid(True, axis='y', linestyle='--', alpha=0.5)

lines = ax.get_lines()
if len(lines) > 1:
    lines[1].set_linestyle('--')

for q, label in zip([13, 26, 39, 52], ['Q1', 'Q2', 'Q3', 'Q4']):
    ax.axvline(x=q, color='gray', linestyle=':', linewidth=1)
    ax.text(q, ax.get_ylim()[1], label, color='gray', ha='center', va='top', fontsize=10)

    ax.set_ylabel('Outbound Passengers')
    ax.set_xlabel('Week of Year')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
for terminal in master_df['terminal'].unique():
    subset = master_df[master_df['terminal'] == terminal]
    avg_by_week = subset.groupby('weekofyear')['occupancy'].mean()
    plt.plot(avg_by_week.index, avg_by_week.values, label=terminal)

plt.xlabel('Week of Year')
plt.ylabel('Average Occupancy')
plt.title('Average Occupancy per Terminal per Week - 2023')
plt.legend()
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.gca().set_ylim(bottom=0)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x):,}"))

for q in [13, 26, 39, 52]:
    plt.axvline(x=q, color='gray', linestyle=':', linewidth=1)
    plt.text(q, plt.gca().get_ylim()[1], f'Q{[13,26,39,52].index(q)+1}', color='gray', ha='center', va='top', fontsize=10)

plt.show()

In [ ]:
ave_occupancy_per_carpark = master_df.groupby(['terminal', 'carpark_name','weekofyear'])['occupancy'].mean().unstack().fillna(0)

ave_occupancy_per_carpark

for terminal in master_df['terminal'].unique():
    subset = master_df[master_df['terminal'] == terminal]
    plt.figure(figsize=(12, 6))
    for carpark in subset['carpark_name'].unique():
        carpark_data = subset[subset['carpark_name'] == carpark].groupby(new_second_ledger_column_time_name)['occupancy'].mean()
        plt.plot(carpark_data.index, carpark_data.values, label=f"{carpark}")
    plt.xlabel('Date')
    plt.ylabel('Occupancy')
    plt.title(f'Occupancy Over Time - Terminal {terminal}')
    plt.legend()
    plt.grid(True)
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x):,}"))
    plt.show()

# 4. Train models

## Split Data

In [ ]:
master_df.columns

In [ ]:
occupancy_view_df = master_df.groupby(['terminal', 'carpark_name']).agg({'occupancy': ['sum', 'mean']})
occupancy_view_df = occupancy_view_df.applymap(lambda x: "{:,}".format(int(x)))
occupancy_view_df

In [ ]:
master_df.groupby(['financial_quarter', 'terminal']).agg({'occupancy': ['sum', 'mean']}).astype(int)

In [ ]:
group_split_list = ['financial_quarter', 'terminal_num', 'carpark_name_num']

if use_weekly:
    print("Using weekly data for train-test split...")
    master_df_sorted = weekly_df.sample(frac=1).reset_index(drop=True)
else:
    # Sort by date
    master_df_sorted = master_df.drop(columns=columns_to_encode).sample(frac=1).reset_index(drop=True)

group_indices_series = master_df_sorted.groupby(group_split_list).ngroup()

if naive_split_bool:
    # 70/30 split
    split_idx = int(len(master_df_sorted) * 0.7)
    train_df = master_df_sorted.iloc[:split_idx]
    test_df = master_df_sorted.iloc[split_idx:]

else:
    print("Using GroupShuffleSplit for train-test split...")
    gss = GroupShuffleSplit(n_splits=2, test_size=0.5, random_state=42)

    for train_idx, test_idx in gss.split(master_df_sorted, groups = group_indices_series):
        train_df = master_df_sorted.iloc[train_idx]
        test_df = master_df_sorted.iloc[test_idx]

# Feature train target train, feature test target test
X_train = train_df.drop(columns=['occupancy'])
y_train = train_df['occupancy']
X_test = test_df.drop(columns=['occupancy'])
y_test = test_df['occupancy']

## Trial Models

In [ ]:
# Fit model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict
y_pred_lr = lr.predict(X_test)

# Metrics
r2_score_value_lr = r2_score(y_test, y_pred_lr)
rmse_value_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_value_lr = mean_absolute_error(y_test, y_pred_lr)
mse_value_lr = mean_squared_error(y_test, y_pred_lr)

print(f"R2 Score: {r2_score_value_lr:.4f}")
print(f"RMSE: {rmse_value_lr:.2f}")
print(f"MAE: {mae_value_lr:.2f}")
print(f"MSE: {mse_value_lr:.2f}")

In [ ]:
# Prepare target series for ARIMA
train_target = train_df['occupancy']
test_target = test_df['occupancy']

# Fit ARIMA model (simple order, can be tuned)
arima_model = ARIMA(train_target, order=(1,1,1))
arima_result = arima_model.fit()

# Forecast
forecast = arima_result.forecast(steps=len(test_target))
y_pred_arima = forecast.values
y_test_arima = test_target.values

# Metrics
r2_arima = r2_score(y_test_arima, y_pred_arima)
rmse_arima = np.sqrt(mean_squared_error(y_test_arima, y_pred_arima))
mae_arima = mean_absolute_error(y_test_arima, y_pred_arima)
mse_arima = mean_squared_error(y_test_arima, y_pred_arima)

print(f"ARIMA R2 Score: {r2_arima:.4f}")
print(f"ARIMA RMSE: {rmse_arima:.2f}")
print(f"ARIMA MAE: {mae_arima:.2f}")
print(f"ARIMA MSE: {mse_arima:.2f}")

In [ ]:
use_all = False
no_cv_bool = True
cv_folds = 3

# Compute correlation of features with target
# Ensure feature_columns is defined and accessible
feature_columns = [col for col in train_df.columns if col not in ['occupancy']]

corrs = train_df[feature_columns + ['occupancy']].corr()['occupancy'].drop('occupancy')
corrs_abs = corrs.abs().sort_values(ascending=False)

# Select top N features (e.g., top 8)
top_n = 8
selected_features = corrs_abs.head(top_n).index.tolist()
print("Selected features:", selected_features)

if use_all:
    selected_features = feature_columns

if no_cv_bool:
    cv_folds=[(slice(None), slice(None))]

# Use selected features for training
X_train_corr = train_df[selected_features].fillna(0)
X_test_corr = test_df[selected_features].fillna(0)

# Fit model
rf_corr = RandomForestRegressor(random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [10, 20, 100, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 10],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(rf_corr, param_grid, cv=cv_folds, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train_corr, y_train)

rf_corr = grid_search.best_estimator_
rf_corr.fit(X_train_corr, y_train)

# Predict
y_pred_rf_corr = rf_corr.predict(X_test_corr)

# Metrics
r2_rf_corr = r2_score(y_test, y_pred_rf_corr)
rmse_rf_corr = np.sqrt(mean_squared_error(y_test, y_pred_rf_corr))
mae_rf_corr = mean_absolute_error(y_test, y_pred_rf_corr)
mse_rf_corr = mean_squared_error(y_test, y_pred_rf_corr)

print(f"Random Forest (Corr) R2 Score: {r2_rf_corr:.4f}")
print(f"Random Forest (Corr) RMSE: {rmse_rf_corr:.2f}")
print(f"Random Forest (Corr) MAE: {mae_rf_corr:.2f}")
print(f"Random Forest (Corr) MSE: {mse_rf_corr:.2f}")

In [ ]:
feature_importances = rf_corr.feature_importances_

# Create a DataFrame to display feature importances
importances_df = pd.DataFrame({
    'Feature': X_train_corr.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Print the feature importances
print("Feature Importances:")
print(importances_df)

# Plotting feature importances
plt.figure(figsize=(10, 6))
plt.barh(importances_df['Feature'], importances_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Random Forest - Feature Importances')
plt.gca().invert_yaxis()  # Invert y-axis for better readability
plt.show()

In [ ]:
# Use all features except 'occupancy'
all_features = [col for col in train_df.columns if col != 'occupancy']
X_train_pca = train_df[all_features].fillna(0)
X_test_pca = test_df[all_features].fillna(0)

pca_svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('svr', SVR(kernel='rbf', C=1.0, epsilon=0.2))
])

param_grid = {
    'pca__n_components': list(range(2, min(len(all_features), 10))),
    'svr__C': [0.1, 1.0, 10.0],
    'svr__epsilon': [0.1, 0.2, 0.5]
}

grid_search_pca_svm = GridSearchCV(
    pca_svm_pipeline,
    param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

grid_search_pca_svm.fit(X_train_pca, y_train)
pca_svm_pipeline = grid_search_pca_svm.best_estimator_
print("Best params:", grid_search_pca_svm.best_params_)

pca_svm_pipeline.fit(X_train_pca, y_train)
y_pred_pca_svm = pca_svm_pipeline.predict(X_test_pca)

# Metrics
r2_pca_svm = r2_score(y_test, y_pred_pca_svm)
rmse_pca_svm = np.sqrt(mean_squared_error(y_test, y_pred_pca_svm))
mae_pca_svm = mean_absolute_error(y_test, y_pred_pca_svm)
mse_pca_svm = mean_squared_error(y_test, y_pred_pca_svm)

print(f"PCA+SVM R2 Score: {r2_pca_svm:.4f}")
print(f"PCA+SVM RMSE: {rmse_pca_svm:.2f}")
print(f"PCA+SVM MAE: {mae_pca_svm:.2f}")
print(f"PCA+SVM MSE: {mse_pca_svm:.2f}")

In [ ]:
model_name = 'xgb_grid_search'

cross_validate = False

if cross_validate:
    cv_folds = 3
else:
    cv_folds = [(slice(None), slice(None))]


# pipeline
xgb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    (f'{model_name}', XGBRegressor(random_state=42, n_jobs=-1))
])

# parameter grid (all combinations tested)
param_grid = {
    f'{model_name}__n_estimators': [100, 300, 500],
    f'{model_name}__max_depth': [3, 6, 9],
    f'{model_name}__learning_rate': [0.01, 0.05, 0.1],
    f'{model_name}__subsample': [0.8, 1.0],
    f'{model_name}__colsample_bytree': [0.8, 1.0],
}

xgb_pipeline = GridSearchCV(
    estimator=xgb_pipeline,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',  # or r2, etc.
    cv=cv_folds,
    verbose=2,
    n_jobs=-1
)

xgb_pipeline.fit(X_train, y_train)

y_pred_xgb_array = xgb_pipeline.predict(X_test)

xgb_ledger_df = pd.concat(
	[X_test.copy(deep=True), 
    pd.DataFrame({'model_prediction': y_pred_xgb_array}
    , index=X_test.index)],
	axis=1
)

xgb_ledger_df['mae'] = np.abs(xgb_ledger_df['model_prediction'] - y_test)
xgb_ledger_df['mse'] = (xgb_ledger_df['model_prediction'] - y_test)**2
xgb_ledger_df['rmse'] = np.sqrt(xgb_ledger_df['mse'])
xgb_ledger_df['r2'] = 1 - (xgb_ledger_df['mse'] / np.var(y_test))

xgb_ledger_df = pd.concat(
    [
        xgb_ledger_df,
        y_test
    ],
    axis=1
).rename(columns={'occupancy':'ground_truth'})

bins = [-np.inf, -15, -5, 5, 15, np.inf]
accuracy_labels = ['Extreme Underforecast', 'Underforecast', 'Correct', 'Overforecast', 'Extreme Overforecast']
xgb_ledger_df['raw_difference'] = xgb_ledger_df['model_prediction'] - xgb_ledger_df['ground_truth']
xgb_ledger_df['perc_difference'] = xgb_ledger_df['raw_difference'] / xgb_ledger_df['ground_truth'] * 100

xgb_ledger_df['accuracy_bin'] = pd.cut(xgb_ledger_df['perc_difference'], bins=bins, labels=accuracy_labels)

# Metrics
r2_xgb = r2_score(y_test, y_pred_xgb_array)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb_array))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb_array)
mse_xgb = mean_squared_error(y_test, y_pred_xgb_array)

print(f"XGBoost R2 Score: {r2_xgb:.4f}")
print(f"XGBoost RMSE: {rmse_xgb:.2f}")
print(f"XGBoost MAE: {mae_xgb:.2f}")
print(f"XGBoost MSE: {mse_xgb:.2f}")

print("Best params (Grid):", xgb_pipeline.best_params_)
print("Best score (Grid):", xgb_pipeline.best_score_)

In [ ]:
calcs_dict = {
    'mae': 'mean',
    'mse': 'mean',
    'rmse': 'mean',
    'r2': 'mean',
    'model_prediction':'count'
}

xgb_result_df_overall = xgb_ledger_df.groupby(['terminal_num']).agg({**calcs_dict}).reset_index()

xgb_result_df_overall['carpark_name_num'] = 'All Terminals'

xgb_result_df_general = xgb_ledger_df.groupby(['terminal_num', 'carpark_name_num']).agg({**calcs_dict}).reset_index()
xgb_result_df_general['carpark_name_num'] += 1

xgb_result_financial_df = xgb_ledger_df.groupby(['financial_quarter', 'terminal_num', 'carpark_name_num']).agg({**calcs_dict}).reset_index()
xgb_result_financial_df['carpark_name_num'] += 1
xgb_result_financial_df['terminal_num'] += 1

xgb_result_df = pd.concat([xgb_result_df_overall, xgb_result_df_general, xgb_result_financial_df]
                          ).fillna({'financial_quarter': 'Year Round'}
                                   ).rename(
                                       columns={'terminal_num': 'Terminal Number',
                                                 'carpark_name_num': 'Carpark Name', 
                                                 'financial_quarter': 'Financial Quarter', 
                                                 'model_prediction': 'Records Used'})
xgb_result_df['Terminal Number'] += 1
xgb_result_df.head()

accuracy_bin_zip = {'accuracy_bin': 'value_counts'}

# Combine two dictionaries
combined_dict = {**accuracy_bin_zip, **calcs_dict}
xgb_result_df

In [ ]:
xgb_ledger_df.groupby('weekofyear').agg({
    'mae': 'mean',
    
}).plot(kind='line')

In [ ]:
accuracy_bin_df = xgb_ledger_df.groupby(['financial_quarter', 'terminal_num', 'carpark_name_num']).agg(**{
    'Record Count': ('accuracy_bin', 'value_counts')
})

accuracy_bin_df['Percentage'] = accuracy_bin_df.groupby(['financial_quarter', 'terminal_num', 'carpark_name_num'])['Record Count'].transform(lambda x: x / x.sum() * 100)
accuracy_bin_df['Percentage'] = accuracy_bin_df['Percentage'].replace(np.nan, np.inf)

columns_to_agg = ['financial_quarter', 'terminal_num', 'carpark_name_num']
to_append =  accuracy_bin_df.reset_index().groupby(['accuracy_bin', 'financial_quarter']).agg({'Record Count': 'sum', 'Percentage': 'mean'}).reset_index()
# to_append['terminal_num'] += 1
# to_append['carpark_name_num'] += 1
to_append

In [ ]:
# Prepare data (use all features except 'occupancy')
nn_features = [col for col in X_train.columns if col != 'occupancy']
scaler = StandardScaler()
X_train_nn = scaler.fit_transform(X_train[nn_features].fillna(0))
X_test_nn = scaler.transform(X_test[nn_features].fillna(0))

# Build neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_nn.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.1),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Early stopping for best results
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit model
history = model.fit(
    X_train_nn, y_train.values,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=0
)

# Predict
y_pred_nn = model.predict(X_test_nn).flatten()

# Metrics
r2_nn = r2_score(y_test, y_pred_nn)
rmse_nn = mean_squared_error(y_test, y_pred_nn, squared=False)
mae_nn = mean_absolute_error(y_test, y_pred_nn)
mse_nn = mean_squared_error(y_test, y_pred_nn)

print(f"Neural Network R2 Score: {r2_nn:.4f}")
print(f"Neural Network RMSE: {rmse_nn:.2f}")
print(f"Neural Network MAE: {mae_nn:.2f}")
print(f"Neural Network MSE: {mse_nn:.2f}")

# Plot training and validation losses
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Neural Network Training & Validation Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
metrics = {
    'Model': ['Linear Regression', 'Random Forest (Corr)', 'XGBoost', 'ARIMA', 'PCA+SVM'],
    'R2 Score': [r2_score_value_lr, r2_rf_corr, r2_xgb, r2_arima, r2_pca_svm],
    'RMSE': [rmse_value_lr, rmse_rf_corr, rmse_xgb, rmse_arima, rmse_pca_svm],
    'MAE': [mae_value_lr, mae_rf_corr, mae_xgb, mae_arima, mae_pca_svm],
    'MSE': [mse_value_lr, mse_rf_corr, mse_xgb, mse_arima, mse_pca_svm]
}

metrics_df = pd.DataFrame(metrics)



# 5. Display results

In [ ]:
# Plotting the metrics, excluding ARIMA
metrics_df_no_arima = metrics_df[metrics_df['Model'] != 'ARIMA'].copy()

# Rename Linear Regression to (Baseline) Linear Regression
metrics_df_no_arima.loc[metrics_df_no_arima['Model'] == 'Linear Regression', 'Model'] = '(Baseline) Linear Regression'

# Use a color palette for bars
bar_colors = list(mcolors.TABLEAU_COLORS.values())

metrics_to_plot = ['R2 Score', 'RMSE', 'MAE', 'MSE']
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
for ax, metric in zip(axes.flatten(), metrics_to_plot):
    bars = metrics_df_no_arima.plot(
        x='Model', y=metric, kind='barh', ax=ax, legend=False, color=bar_colors[:len(metrics_df_no_arima)]
    )
    ax.set_title(f'{metric} Comparison (Excluding ARIMA)')
    ax.set_ylabel('Model')
    ax.set_xlabel(metric)
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)
    for x in ax.get_xticks():
        ax.axvline(x=x, color='lightgray', linestyle=':', linewidth=0.7, zorder=0)

plt.tight_layout()
plt.show()

# Plotting ARIMA separately
metrics_arima = metrics_df[metrics_df['Model'] == 'ARIMA'].copy()

fig, axes = plt.subplots(1, len(metrics_to_plot), figsize=(15, 5))
for ax, metric in zip(axes, metrics_to_plot):
    ax.barh(metrics_arima['Model'], metrics_arima[metric], color=bar_colors[0])
    ax.set_title(f'ARIMA - {metric}')
    ax.set_ylabel('Model')
    ax.set_xlabel(metric)
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)
    for x in ax.get_xticks():
        ax.axvline(x=x, color='lightgray', linestyle=':', linewidth=0.7, zorder=0)

plt.tight_layout()
plt.show()

In [ ]:
# Rename Linear Regression to (Baseline) Linear Regression
metrics_df_plot = metrics_df.copy()
metrics_df_plot.loc[metrics_df_plot['Model'] == 'Linear Regression', 'Model'] = '(Baseline) Linear Regression'

# Use a color palette for bars
bar_colors = list(mcolors.TABLEAU_COLORS.values())

metrics_to_plot = ['R2 Score', 'RMSE', 'MAE', 'MSE']
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
for ax, metric in zip(axes.flatten(), metrics_to_plot):
    bars = metrics_df_plot.plot(
        x='Model', y=metric, kind='barh', ax=ax, legend=False, color=bar_colors[:len(metrics_df_plot)]
    )
    ax.set_title(f'{metric} Comparison')
    ax.set_ylabel('Model')
    ax.set_xlabel(metric)
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)
    for x in ax.get_xticks():
        ax.axvline(x=x, color='lightgray', linestyle=':', linewidth=0.7, zorder=0)

plt.tight_layout()
plt.show()


In [ ]:
# Filter metrics_df to display only Linear Regression, XGBoost, and Random Forest (Corr)
metrics_df_selected = metrics_df[metrics_df['Model'].isin(['Linear Regression', 'XGBoost', 'Random Forest (Corr)'])].copy()

# Rename Linear Regression to (Baseline) Linear Regression for clarity
metrics_df_selected.loc[metrics_df_selected['Model'] == 'Linear Regression', 'Model'] = '(Baseline) Linear Regression'

# Use a color palette for bars
bar_colors = list(mcolors.TABLEAU_COLORS.values())

metrics_to_plot = ['R2 Score', 'RMSE', 'MAE', 'MSE']
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
for ax, metric in zip(axes.flatten(), metrics_to_plot):
    bars = metrics_df_selected.plot(
        x='Model', y=metric, kind='barh', ax=ax, legend=False, color=bar_colors[:len(metrics_df_selected)]
    )
    ax.set_title(f'{metric} Comparison')
    ax.set_ylabel('Model')
    ax.set_xlabel(metric)
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)
    for x in ax.get_xticks():
        ax.axvline(x=x, color='lightgray', linestyle=':', linewidth=0.7, zorder=0)

plt.tight_layout()
plt.show()

# END - Save report

In [ ]:
# Create a new Excel file and add worksheets for each graphic
output = io.BytesIO()
workbook = xlsxwriter.Workbook(output, {'in_memory': True})

# 1. Weekly Passenger Count Over Time
ws_passenger = workbook.add_worksheet('Weekly Passenger Count')
fig1, ax1 = plt.subplots(figsize=(12,6))
sliced_flights_df.groupby(['weekofyear','terminal_name']).agg({'pax_quantity': 'sum'}).unstack().droplevel(level=0, axis=1).plot(
    kind='line', title='Weekly Passenger Count Over Time', ax=ax1
)
ax1.set_ylim(bottom=0)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x):,}"))
ax1.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
imgdata1 = io.BytesIO()
fig1.savefig(imgdata1, format='png')
imgdata1.seek(0)
ws_passenger.insert_image('B2', 'weekly_passenger.png', {'image_data': imgdata1})
plt.close(fig1)

# 2. Average Occupancy per Terminal per Week
ws_occupancy = workbook.add_worksheet('Terminal Occupancy')
fig2 = plt.figure(figsize=(14, 6))
for terminal in master_df['terminal'].unique():
    subset = master_df[master_df['terminal'] == terminal]
    avg_by_week = subset.groupby('weekofyear')['occupancy'].mean()
    plt.plot(avg_by_week.index, avg_by_week.values, label=terminal)
plt.xlabel('Week of Year')
plt.ylabel('Average Occupancy')
plt.title('Average Occupancy per Terminal per Week - 2023')
plt.legend()
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.gca().set_ylim(bottom=0)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x):,}"))
plt.close(fig2)
imgdata2 = io.BytesIO()
fig2.savefig(imgdata2, format='png')
imgdata2.seek(0)
ws_occupancy.insert_image('B2', 'terminal_occupancy.png', {'image_data': imgdata2})

# 3. Occupancy Over Time - Terminal/Carpark
ws_carpark = workbook.add_worksheet('Carpark Occupancy')
fig3_list = []
for terminal in master_df['terminal'].unique():
    fig3 = plt.figure(figsize=(12, 6))
    for carpark in master_df[master_df['terminal'] == terminal]['carpark_name'].unique():
        carpark_data = master_df[(master_df['terminal'] == terminal) & (master_df['carpark_name'] == carpark)].groupby(new_second_ledger_column_time_name)['occupancy'].mean()
        plt.plot(carpark_data.index, carpark_data.values, label=f"{carpark}")
    plt.xlabel('Date')
    plt.ylabel('Occupancy')
    plt.title(f'Occupancy Over Time - Terminal {terminal}')
    plt.legend()
    plt.grid(True)
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{int(x):,}"))
    imgdata3 = io.BytesIO()
    fig3.savefig(imgdata3, format='png')
    imgdata3.seek(0)
    ws_carpark.insert_image(f'B{2 + 20 * list(master_df["terminal"].unique()).index(terminal)}', f'carpark_{terminal}.png', {'image_data': imgdata3})
    plt.close(fig3)

# 4. Feature Importances
ws_importance = workbook.add_worksheet('Feature Importances')
fig4 = plt.figure(figsize=(10, 6))
plt.barh(importances_df['Feature'], importances_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Random Forest - Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
imgdata4 = io.BytesIO()
fig4.savefig(imgdata4, format='png')
imgdata4.seek(0)
ws_importance.insert_image('B2', 'feature_importances.png', {'image_data': imgdata4})
plt.close(fig4)

# 5. Model Metrics Comparison
ws_metrics = workbook.add_worksheet('Model Metrics')
fig5, axes = plt.subplots(2, 2, figsize=(15, 10))
metrics_df_plot = metrics_df.copy()
metrics_df_plot.loc[metrics_df_plot['Model'] == 'Linear Regression', 'Model'] = '(Baseline) Linear Regression'
bar_colors = list(mcolors.TABLEAU_COLORS.values())
metrics_to_plot = ['R2 Score', 'RMSE', 'MAE', 'MSE']
for ax, metric in zip(axes.flatten(), metrics_to_plot):
    metrics_df_plot.plot(
        x='Model', y=metric, kind='barh', ax=ax, legend=False, color=bar_colors[:len(metrics_df_plot)]
    )
    ax.set_title(f'{metric} Comparison')
    ax.set_ylabel('Model')
    ax.set_xlabel(metric)
    ax.grid(True, axis='y', linestyle='--', alpha=0.5)
    for x in ax.get_xticks():
        ax.axvline(x=x, color='lightgray', linestyle=':', linewidth=0.7, zorder=0)
plt.tight_layout()
imgdata5 = io.BytesIO()
fig5.savefig(imgdata5, format='png')
imgdata5.seek(0)
ws_metrics.insert_image('B2', 'model_metrics.png', {'image_data': imgdata5})
plt.close(fig5)

# xgb_result_df

workbook.close()
output.seek(0)

with open(path_to_report, 'wb') as f:
    f.write(output.read())

